In [1]:
# import needed libraries 
import pandas as pd
import numpy as np
import random
import json

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def split_in_two(df, max_captions_per_image):
  '''
  Slit_in_two split a dataframe with columns 'caption' and 
  'image_id' which into 2 dataframes of the same dimension, which contains respectively a
  correct couple (image_id, caption) and wrong couple (image_id, caption)

  Label (third column in the final dataframes): 1:correct couple  0: wrong couple
  max_captions_per_image: is the max possible number of captions for a single image

  '''
  meta=int(df['image_id'].nunique()/2)
  count=0
  id_immagini=df['image_id'].unique()
  dataset_p=pd.DataFrame(columns=["caption","image_id"])
  dataset_n=pd.DataFrame(columns=["caption","image_id"])

  for img_id in id_immagini:

    if (count<meta):
      new_row=df[df['image_id']==img_id].iloc[0]
      dataset_p=dataset_p.append(new_row,ignore_index=True)
      count+=1

    if (count>=meta) and count < (2*meta):
      n=random.randint(0,(df['image_id'].nunique()-max_captions_per_image))
      caption=df[df['image_id']!=img_id].iloc[n].caption
      new_row={'caption':caption,'image_id':img_id}
      dataset_n=dataset_n.append(new_row,ignore_index=True)
      count+=1

  dataset_n['Label']=[0]*len(dataset_n)
  dataset_p['Label']=[1]*len(dataset_p)
  dataset_p['image_id'].apply(lambda id: int(id))

  return dataset_p, dataset_n

In [ ]:
#read json dataset
train_j = json.load(open("/content/drive/captions_train2014.json")) 
validation_j = json.load(open("/content/drive/captions_val2014.json")) 

In [ ]:
#create dataframes
valid = pd.DataFrame(validation_j["annotations"]) 
train_annotations= pd.DataFrame(train_j["annotations"]) 
valid = valid[["caption", "image_id"]]
train = train_annotations[["caption", "image_id"]] 

In [ ]:
#create dataframes: _n for negative labels, _p for positive ones
validation_p, validation_n = split_in_two(valid,7)
train_p, train_n  = split_in_two(train,7)

In [ ]:
#merge the positive dataset with the negative dataset
validation_np= validation_n.merge(validation_p, how="outer")
train_np = train_n.merge(train_p, how="outer")

In [ ]:
validation_np.to_csv("/content/drive/dataval_n.csv", index=False)
train_np.to_csv("/content/drive/trainval_p.csv", index=False)  